# Setup

In [ ]:
DESIRED_IMG_WIDTH=1024

In [ ]:
from pathlib import Path
import pylibjpeg
import gdcm
import pydicom
import fastcore.xtras
import PIL
from fastai.medical.imaging import *
from tqdm.auto import tqdm

# Find Images

In [ ]:
input_path = Path('train_images')
output_path = Path('train_pngs_benchmarking')
output_path.mkdir(parents=True, exist_ok=True)

In [ ]:
input_path.ls()

(#27) [Path('train_images/21221'),Path('train_images/19334'),Path('train_images/26810'),Path('train_images/10514'),Path('train_images/17064'),Path('train_images/46113'),Path('train_images/58906'),Path('train_images/62679'),Path('train_images/27507'),Path('train_images/24167')...]

In [ ]:
dcms = get_dicom_files(input_path)
dcms

(#128) [Path('train_images/21221/448741934.dcm'),Path('train_images/21221/1064888428.dcm'),Path('train_images/21221/282742387.dcm'),Path('train_images/21221/1236706393.dcm'),Path('train_images/21221/1580909504.dcm'),Path('train_images/21221/1821800285.dcm'),Path('train_images/19334/102489772.dcm'),Path('train_images/19334/949176137.dcm'),Path('train_images/19334/375445182.dcm'),Path('train_images/19334/2122421722.dcm')...]

# Method 1
Found while working on SIIM Covid-19 project.

In [ ]:
# From https://www.kaggle.com/xhlulu/siim-covid-19-convert-to-jpg-256px
# https://www.kaggle.com/code/stephenhudson/siim-covid-19-1-data-prep-test-data/notebook

import numpy as np
import pydicom
import gdcm
from pydicom.pixel_data_handlers.util import apply_voi_lut
from PIL import Image

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
            
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

# from pqdm.processes import pqdm

def resize_and_save(input_path, output_path, size, sub_dir='train_images'):
    img = read_xray(input_path)
    size2 = max(size, round(size*img.shape[0]/img.shape[1]))
    img2 = resize(img, size=size2, keep_ratio=True)
    index = input_path.parts.index(sub_dir)
    png = output_path.joinpath(*input_path.parts[index:]).with_suffix('.png')
    png.parent.mkdir(parents=True, exist_ok=True)
    img2.save(png)

In [ ]:
PIL.__version__

'9.4.0'

In [ ]:
%%timeit
for dcm in tqdm(dcms, desc='Rescale training images'):
    resize_and_save(dcm, output_path, DESIRED_IMG_WIDTH)

Rescale training images:   0%|          | 0/128 [00:00<?, ?it/s]

Rescale training images:   0%|          | 0/128 [00:00<?, ?it/s]

Rescale training images:   0%|          | 0/128 [00:00<?, ?it/s]

Rescale training images:   0%|          | 0/128 [00:00<?, ?it/s]

Rescale training images:   0%|          | 0/128 [00:00<?, ?it/s]

Rescale training images:   0%|          | 0/128 [00:00<?, ?it/s]

Rescale training images:   0%|          | 0/128 [00:00<?, ?it/s]

Rescale training images:   0%|          | 0/128 [00:00<?, ?it/s]

2min 36s ± 1.31 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
